In [2]:
import pandas as pd
from sklearn import linear_model

In [3]:
url = 'https://github.com/alinaprolygina/BDinMED/raw/main/HW4/DataOnly/WaitData.Published.xlsx'
data = pd.read_excel(url, sheet_name='F3')

In [4]:
data.head()

,Wait,x_ArrivalDTTM,x_ScheduledDTTM,x_BeginDTTM,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,0,2025-08-02 16:47:44,2025-08-02 17:45:00,2025-08-02 17:45:03,372,62,6,249,6,5,...,4,2,0,0,0,0,3,1,1,0
1,-3,2025-08-02 16:38:46,2025-08-02 17:15:00,2025-08-02 17:12:14,335,67,3,195,5,4,...,4,1,2,0,32,0,2,1,2,0
2,-25,2025-08-02 16:29:25,2025-08-02 17:30:00,2025-08-02 17:05:15,274,68,4,150,4,5,...,4,1,1,1,13,16,1,1,3,0
3,36,2025-08-02 16:16:24,2025-08-02 16:45:00,2025-08-02 17:21:55,369,73,2,179,5,6,...,3,1,2,1,45,16,3,1,1,0
4,-21,2025-08-02 16:04:36,2025-08-02 17:15:00,2025-08-02 16:54:49,438,73,5,192,6,5,...,3,0,2,1,32,15,4,0,0,1


In [8]:
data = data.loc[:, ~data.columns.str.startswith('x_')]

In [9]:
data.head()

,Wait,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,LineCount2,LineCount3,LineCount4,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,0,372,62,6,249,6,5,6,6,4,...,4,2,0,0,0,0,3,1,1,0
1,-3,335,67,3,195,5,4,6,6,5,...,4,1,2,0,32,0,2,1,2,0
2,-25,274,68,4,150,4,5,6,4,6,...,4,1,1,1,13,16,1,1,3,0
3,36,369,73,2,179,5,6,4,5,5,...,3,1,2,1,45,16,3,1,1,0
4,-21,438,73,5,192,6,5,5,6,5,...,3,0,2,1,32,15,4,0,0,1


In [10]:
Y = data['Wait']
X = data.drop(columns=['Wait'])

In [12]:
model = linear_model.LinearRegression()
model.fit(X, Y)
Ypred = model.predict(X) # use trained regression model to predict
r = Y-Ypred # compute prediction error (residual)
e = abs(r).mean() # compute model error
median_residual = abs(r).median()

print(f'Model error (e): {e}')
print(f'The median of absolute residual values: {median_residual}')

Model error (e): 23.16628893634719
The median of absolute residual values: 19.126063446073886


In [14]:
# Run Python feature selection
if True: # just in case I want to disable this part
  print('\n>Python feature selection:')
  from sklearn.feature_selection import RFE
  from itertools import compress

  for nFeatures in range(1,4):
    rfe = RFE(model, n_features_to_select=nFeatures)
    X_rfe = rfe.fit_transform(X,Y) #transforming data using RFE
    #Fitting the data to model
    model.fit(X_rfe,Y)
    #print(rfe.support_)
    #print(rfe.ranking_)
    cols = list(compress( X.columns, rfe.support_))
    model.fit(X[cols],Y)
    e = abs(Y-model.predict(X[cols])).mean()
    print(e, cols)


>Python feature selection:
32.50541172120655 ['CardiacCount']
31.22493770698274 ['CardiacCount', 'DelayedInLine']
31.25752894868932 ['CardiacCount', 'StartTime', 'DelayedInLine']


In [25]:
best_feature1 = None
lowest_error1 = float('inf')

for feature in X.columns:
    model.fit(X[[feature]], Y)
    Ypred = model.predict(X[[feature]])
    error = abs(Y - Ypred).mean()

    if error < lowest_error1:
        lowest_error1 = error
        best_feature1 = feature

print(f'Best F1: {best_feature} with error {lowest_error}')

Best F1: DelayedInLine with error 31.227057699733514


In [27]:
best_features_2 = [best_feature]
lowest_error_2 = float('inf')

for feature in X.columns:
    if feature != best_feature:
        temp_features = [best_feature, feature]
        model.fit(X[temp_features], Y)
        Ypred = model.predict(X[temp_features])
        error = abs(Y - Ypred).mean()

        if error < lowest_error_2:
            lowest_error_2 = error
            best_features_2 = temp_features

print(f'Best F1 and F2: {best_features_2} with error {lowest_error_2}')

Best F1 and F2: ['DelayedInLine', 'LineCount0Strict'] with error 30.137965863467567


In [28]:
best_features_3 = best_features_2.copy()
lowest_error_3 = float('inf')

for feature in X.columns:
    if feature not in best_features_2:
        temp_features = best_features_2 + [feature]
        model.fit(X[temp_features], Y)
        Ypred = model.predict(X[temp_features])
        error = abs(Y - Ypred).mean()

        if error < lowest_error_3:
            lowest_error_3 = error
            best_features_3 = temp_features

print(f'Best F3, F4 and F5 {best_features_3} with error {lowest_error_3}')

Best F3, F4 and F5 ['DelayedInLine', 'LineCount0Strict', 'LineCount0'] with error 28.696490632253404


In [38]:
def greedy_feature_selection(X, Y, max_features=15, error_threshold=24):
    best_features = []
    errors = []
    current_error = float('inf')

    for _ in range(max_features):
        lowest_error = float('inf')
        best_feature_to_add = None

        for feature in X.columns:
            if feature not in best_features:
                temp_features = best_features + [feature]
                model.fit(X[temp_features], Y)
                predictions = model.predict(X[temp_features])
                error = abs(Y - predictions).mean()

                if error < lowest_error:
                    lowest_error = error
                    best_feature_to_add = feature

        if best_feature_to_add is not None:
            best_features.append(best_feature_to_add)
            current_error = lowest_error
            errors.append(current_error)
            # print(f'Add: {best_feature_to_add}, error: {current_error}')

            if current_error < error_threshold:
                print(f'Error under {error_threshold} reached with {len(best_features)}')
                break
        else:
            print('All features added')
            break

    return best_features, errors

In [39]:
best_features1, errors1 = greedy_feature_selection(X, Y, max_features=1, error_threshold=24)
best_features2, errors2 = greedy_feature_selection(X, Y, max_features=2, error_threshold=24)
best_features3, errors3 = greedy_feature_selection(X, Y, max_features=3, error_threshold=24)

best_features_opt, errors_opt = greedy_feature_selection(X, Y, max_features=15, error_threshold=24)

print(f'Best 1: {best_features1} with error {errors1}')
print(f'Best 2: {best_features2} with error {errors2}')
print(f'Best 3: {best_features_3} with error {errors3}')

print(f'Optimal {len(best_features_opt)} features {best_features_opt} with error {errors_opt}')

Error under 24 reached with 10
Best 1: ['DelayedInLine'] with error [31.227057699733514]
Best 2: ['DelayedInLine', 'LineCount0Strict'] with error [31.227057699733514, 30.137965863467567]
Best 3: ['DelayedInLine', 'LineCount0Strict', 'LineCount0'] with error [31.227057699733514, 30.137965863467567, 28.696490632253404]
Optimal 10 features ['DelayedInLine', 'LineCount0Strict', 'LineCount0', 'SumDelayWaitingInLine', 'AheadCount', 'NumCompletedToday', 'NumScheduledNextW2', 'DelayCount', 'InProgressSize', 'StartTime4'] with error [31.227057699733514, 30.137965863467567, 28.696490632253404, 28.246309585132344, 28.147006842164537, 24.989574734774788, 24.65918640725134, 24.329071006187295, 24.071936786758204, 23.846509492065334]


In [40]:
features = []

columns = list(X.columns)
for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        for k in range(j + 1, len(columns)):
            F1, F2, F3 = columns[i], columns[j], columns[k]
            model.fit(X[[F1, F2, F3]], Y)
            Ypred = model.predict(X[[F1, F2, F3]])
            error = abs(Y - Ypred).mean()
            features.append((error, F1, F2, F3))

features.sort(key=lambda x: x[0])

print(f'Best model: {features[0]}')
print(f'Top 5')
for m in features[:5]:
    print(m)


Best model: (26.072745167091224, 'AheadCount', 'SumDelayWaitingByExamCode', 'NumCompletedToday')
Top 5
(26.072745167091224, 'AheadCount', 'SumDelayWaitingByExamCode', 'NumCompletedToday')
(26.242252770746937, 'LineCount0', 'AheadCount', 'NumCompletedToday')
(26.252630216305, 'AheadCount', 'NumCompletedToday', 'OutpatientWaitingCount')
(26.35199473233048, 'SumWaits', 'AheadCount', 'NumCompletedToday')
(26.3613855980382, 'AheadCount', 'SumWaitByTaskTypeLine', 'NumCompletedToday')
